# Дектораторы

В этом домашнем задании мы напишем собственные дектораторы, которые будут менять системные объекты. Но для начала мы с вами познакомимся с функцией `write`.

In [1]:
import sys

sys.stdout.write("Hello, my friend!")

Hello, my friend!

17

Это метод объектов file-like классов, то есть классов, которые реализуют семантику "Меня можно создать, из меня можно прочитать и в меня можно записать".

Самый главный пример такого объекта -- объект `file`, являющийся результатом вызова фукнции `open()`. Для простоты и универсальности взаимодействия, стандартный ввод и стандартный вывод тоже являются файлами, из которых можно читать и в которые можно писать. 

In [2]:
output = open("./some_test_data.txt", "w")

In [3]:
output.write("123")

3

In [4]:
output.close()

Как вы могли заметить, функция возвращает число записанных байт. Это важная часть контракта, которую нужно поддержать, если вы хотите как-то подменять эту функцию.

## Задача 1

Для начала, давайте подменим метод `write` у объекта `sys.stdin` на такую функцию, которая перед каждым вызовом оригинальной функции записи данных в `stdout` допечатывает к тексту текущую метку времени.

In [5]:
from datetime import datetime

original_write = sys.stdout.write


def my_write(string_text: str) -> int:
    if string_text == "\n":  # bypass newline
        return original_write("\n")

    date_now = datetime.now().strftime("[%Y-%m-%d %H:%M%:%S]: ")
    return original_write(date_now + string_text)


sys.stdout.write = my_write

In [6]:
print("1, 2, 3")

[2023-12-07 20:07:59]: 1, 2, 3


In [7]:
sys.stdout.write = original_write

Вывод должен был бы быть примерно таким:

```
[2021-12-05 12:00:00]: 1, 2, 3
```

## Задача 2

Упакуйте только что написанный код в декторатор. Весь вывод фукнции должен быть помечен временными метками так, как видно выше.

In [8]:
def timed_output(function):
    original_write = sys.stdout.write

    def wrapper(*args, **kwargs):
        sys.stdout.write = my_write  # очень похоже на патчинг
        result = function(*args, **kwargs)
        sys.stdout.write = original_write
        return result

    return wrapper

In [9]:
@timed_output
def print_greeting(name):
    print(f"Hello, {name}!")

In [35]:
print_greeting("Саша")  # Согласно гугл доку

[2023-12-07 20:14:07]: Hello, Саша!


Вывод должен быть похож на следующий:

```
[2021-12-05 12:00:00]: Hello, Nikita!
```

## Задача 3

Напишите декторатор, который будет перенаправлять вывод фукнции в файл. 

Подсказка: вы можете заменить объект sys.stdout каким-нибудь другим объектом.

In [11]:
def redirect_output(filepath):
    def decorator(function):
        def wrapper(*args, **kwargs):
            original_stdout = sys.stdout
            sys.stdout = open(filepath, "w", encoding="utf-8")
            result = function(*args, **kwargs)
            sys.stdout = original_stdout
            return result

        return wrapper

    return decorator

In [27]:
@redirect_output("./function_output.txt")
def calculate():
    for power in range(1, 5):
        print("|", end="")
        for num in range(1, 20):
            print(f"{num**power:>7,}", end="|")
        print()

In [28]:
calculate()

Я немного разукрасил

In [29]:
%cat function_output.txt

|      1|      2|      3|      4|      5|      6|      7|      8|      9|     10|     11|     12|     13|     14|     15|     16|     17|     18|     19|
|      1|      4|      9|     16|     25|     36|     49|     64|     81|    100|    121|    144|    169|    196|    225|    256|    289|    324|    361|
|      1|      8|     27|     64|    125|    216|    343|    512|    729|  1,000|  1,331|  1,728|  2,197|  2,744|  3,375|  4,096|  4,913|  5,832|  6,859|
|      1|     16|     81|    256|    625|  1,296|  2,401|  4,096|  6,561| 10,000| 14,641| 20,736| 28,561| 38,416| 50,625| 65,536| 83,521|104,976|130,321|


И ни в коем случае не вспоминаем, что в `print` есть параметр, отвечающий за перенаправление выхода в file-like object.

In [33]:
?print

Signature: print(*args, sep=' ', end='\n', file=None, flush=False)
Docstring:
Prints the values to a stream, or to sys.stdout by default.

sep
  string inserted between values, default a space.
end
  string appended after the last value, default a newline.
file
  a file-like object (stream); defaults to the current sys.stdout.
flush
  whether to forcibly flush the stream.
Type:      builtin_function_or_method